# Fluxo Novo

Ordem das células

- Importando base de dados
- Definindo lojas que estarão no relatório
- Definindo dia do relatório
- Definindo emails que receberão o relatório
- Rodando o código para a criação do relatório
- Preparando o html
- Rodando Mock do disparador de email e disparando email

# Definindo Emails e Lojas

In [ ]:
# storesList = ['0502']
# storesList = ['0787']
# storesList = ['0664']
storesList = ['1071']
# storesList = ['10I7']
# storesList = ['0502', '0787', '0664', '1071']
print('lojas', storesList)
reportDate = datetime.datetime.now() - timedelta(days=1)
print('dia escolhido ', reportDate)

## Importando base de dados


In [796]:
# -> Rodar tudo para ter os dataframes corretos
from commom.database.queries.query_lojas import QUERY_LOJAS
from commom.database.queries.query_vendas_nf_novo import QUERY_VENDAS_NF_NOVO
from commom.database.queries.query_metas import QUERY_METAS
from commom.database.queries.query_parcelas import QUERY_PARCELAS
from commom.database.queries.query_vendas_nf import QUERY_VENDAS_NF
from commom.database.queries.query_vendedores import QUERY_VENDEDORES
from commom.database.queries.query_loja_vendedores import QUERY_LOJA_VENDEDORES
from commom.kpi_data.kpi_data_manager import DataHandler, KpiDataManager
from typing import Tuple
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from innovation_messenger import Messenger
import os
from innovation_messenger import IEmailProperties
import missingno as msno

manager = KpiDataManager()
handler = DataHandler()

# Service
## -> inicio
###     -> fetch data from local pickle
local_df_fato_vendas = handler.read_from_local_pickle('df_fato_vendas.pkl')
local_df_lojas = handler.read_from_local_pickle('df_lojas.pkl')
local_df_vendedores = handler.read_from_local_pickle('df_vendedores.pkl')
local_df_parcelas = handler.read_from_local_pickle('df_parcelas.pkl')
local_df_metas = handler.read_from_local_pickle('df_metas.pkl')
local_df_vendedor_loja = handler.read_from_local_pickle('df_vendedor_loja.pkl')

###     -> fetch data from bq
# local_df_fato_vendas = handler.read_from_bigquery(QUERY_VENDAS_NF_NOVO)
# local_df_lojas = handler.read_from_bigquery(QUERY_LOJAS)
# local_df_vendedores = handler.read_from_bigquery(QUERY_VENDEDORES)
# local_df_parcelas = handler.read_from_bigquery(QUERY_PARCELAS)
# local_df_metas = handler.read_from_bigquery(QUERY_METAS).rename(
#     columns={"cpf_vendedor": "cpf_vendedor_inteiro", "loja": "distributorId"}
# )
# local_df_vendedor_loja = handler.read_from_bigquery(QUERY_LOJA_VENDEDORES)



In [801]:
### - Funções base [cria df_vendas_pdv, cria mascaras(vendas_loja | vendas_vendedor | meta), faz o merge(vendas-CPFvendedor e CPFvendedor-nome) ]

def _abreviate_vendedor_name(full_name: str) -> str:
    full_name = full_name.replace("   ", " ")
    full_name = full_name.replace("  ", " ")

    if len(full_name.title().split(" ")) > 1:
        splited_name = full_name.title().split(" ")
        nome_completo = splited_name[0]
        splited_name.pop(0)

        for name in splited_name:
            nome_completo += "(@!_)" + name[0] + "."

        return nome_completo

    return full_name

# ----------------------------------------
# ----------------------------------------

def create_vendas_pdv(df_original_fato_vendas: pd.DataFrame):
    PDV_TYPE_LIST = ['Propria']
    maskPropria = df_original_fato_vendas["type"].isin(PDV_TYPE_LIST)
    
    maksNullLoja = df_original_fato_vendas['loja_esforco'].isnull()
    maksNullDistributor = df_original_fato_vendas['distributorId'].isnull()

    df_vendas_pdvLoja = df_original_fato_vendas[maskPropria & ~maksNullLoja].copy()
    df_vendas_pdvDistributor = df_original_fato_vendas[maskPropria & ~maksNullDistributor].copy()
    
    df_vendas_pdvLoja['createdAt'] = pd.to_datetime(df_vendas_pdvLoja['createdAt'])
    df_vendas_pdvDistributor['createdAt'] = pd.to_datetime(df_vendas_pdvDistributor['createdAt'])
    
    renameDict = {
        'displayCode': 'displaycode',
        'vendedor_esforco': 'cpf_vendedor_inteiro',
        'esforco_data': 'invoice_date'
    }

    df_vendas_pdvLoja = df_vendas_pdvLoja.reset_index(drop=True)
    df_vendas_pdvDistributor = df_vendas_pdvDistributor.reset_index(drop=True)
    
    df_vendas_pdvLoja['year'] = df_vendas_pdvLoja['createdAt'].dt.year
    df_vendas_pdvLoja['month'] = df_vendas_pdvLoja['createdAt'].dt.month
    df_vendas_pdvLoja['day'] = df_vendas_pdvLoja['createdAt'].dt.day
    
    df_vendas_pdvDistributor['year'] = df_vendas_pdvDistributor['createdAt'].dt.year
    df_vendas_pdvDistributor['month'] = df_vendas_pdvDistributor['createdAt'].dt.month
    df_vendas_pdvDistributor['day'] = df_vendas_pdvDistributor['createdAt'].dt.day
    
    df_vendas_pdvLoja.rename(columns=renameDict, inplace=True)
    df_vendas_pdvDistributor.rename(columns=renameDict, inplace=True)
    
    return df_vendas_pdvLoja, df_vendas_pdvDistributor

# ----------------------------------------
# ----------------------------------------

def merge_kpiVendedor_nome(df_vendedores:pd.DataFrame, vendedor_loja:pd.DataFrame, df_kpiVendedor_filtro: pd.DataFrame, id_loja: str) -> pd.DataFrame:
    df_nome_vendedor = df_vendedores[["name", "cpf"]].copy()
    df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
    df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)
    
    maskvendedorLoja = vendedor_loja['distributorId'] == id_loja
    vendedor_loja = vendedor_loja[maskvendedorLoja].copy()
    listCpf = vendedor_loja['employeeCpf'].tolist()
    
    maskCpf = df_kpiVendedor_filtro['cpf_vendedor_inteiro'].isin(listCpf)
    df_kpiVendedor_filtro = df_kpiVendedor_filtro[maskCpf]
    
    df_kpiVendedor_filtro = df_nome_vendedor.merge(df_kpiVendedor_filtro, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )
    
    return df_kpiVendedor_filtro

def build_vendedor():
    df_nome_vendedor = local_df_vendedores[["name", "cpf"]]
    df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
    df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)
    return df_nome_vendedor

# ----------------------------------------
# ----------------------------------------

def create_masks_meta_vendedor(periodo:str, df_vendas_:pd.DataFrame , df_metas:pd.DataFrame, date: datetime.datetime):
    mask_meta = df_metas["tipo_meta"] == "Meta Vendedor PDV"
    mask_vendas = pd.Series(np.ones(df_vendas_.shape[0]).astype(bool))
    
    if periodo == "Dia":
        mask_yesterday_vendas = df_vendas_["invoice_date"] == date
        mask_vendas = mask_vendas & mask_yesterday_vendas
        
        mask_metas_yesterday = df_metas["date"] == date
        mask_meta = mask_meta & mask_metas_yesterday

    return mask_meta, mask_vendas

# ----------------------------------------
# ----------------------------------------

def create_masks_meta_loja(periodo:str,  df_vendas_:pd.DataFrame , df_metas:pd.DataFrame, date: datetime.datetime):
    mask_meta = df_metas["tipo_meta"] == "Meta Loja PDV"
    mask_vendas = pd.Series(np.ones(df_vendas_.shape[0]).astype(bool))
    
    if periodo == "Dia":
        mask_yesterday_vendas = df_vendas_["invoice_date"] == date
        mask_vendas = mask_vendas & mask_yesterday_vendas
        
        mask_metas_yesterday = df_metas["date"] == date
        mask_meta = mask_meta & mask_metas_yesterday

    return mask_meta, mask_vendas


In [802]:
### - Função para criar o df_kpi_vendedor
def create_kpi_vendedor_df(kpi_period_:str, df_metas: pd.DataFrame, date: datetime.datetime, df_vendas_pdv_: pd.DataFrame) -> pd.DataFrame:
    maskmeta, maskvenda = create_masks_meta_vendedor(periodo=kpi_period_, df_vendas_=df_vendas_pdv_, df_metas=df_metas, date=date)
    AGG_DICT = { 
        "discount": "sum", 
        "netValue": "sum", 
        "cost": "sum" 
    }

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["loja_esforco", "year","cpf_vendedor_inteiro"],
        "Mês": ["loja_esforco", "year", "month", "cpf_vendedor_inteiro"],
        "Dia": ["loja_esforco", "year", "month", "day", "cpf_vendedor_inteiro"],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period_]

    df_kpis: pd.DataFrame = df_vendas_pdv_[maskvenda].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["fiscalsParentOrder"].nunique().reset_index().rename(columns={"fiscalsParentOrder": "tkts"}),
        how="left",
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    metas_df = df_metas[maskmeta].copy()
    metas_df.rename(columns={"distributorId": "loja_esforco"}, inplace=True)
    
    # dfmodificado = df_kpis.merge(
    #     metas_df,
    #     how="left"
    # )
    
    df_kpis = df_kpis.merge(
        metas_df.groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_.groupby(group_by_columns)["invoice_date"]
        .nunique()
        .reset_index()
        .rename(columns={"invoice_date": "days_of_work"}),
        how="left",
    )

    if kpi_period_ == "Dia":
        df_kpis["Dia"] = reportDate.day

    if kpi_period_ == "Mês":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
    else:
        df_kpis["tkts_per_day"] = df_kpis["tkts"]

    df_kpis["type"] = kpi_period_

    df_kpis["mkp"] = (df_kpis["netValue"] / df_kpis["cost"]).round(2)

    df_kpis["tkt_medio"] = df_kpis["netValue"] / df_kpis["tkts"].round(2)
    df_kpis["meta_percentual"] = (df_kpis["netValue"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount"] / df_kpis["netValue"]) * 100).round(2)
    
    df_kpis.replace({'mkp': [np.inf, -np.inf]}, 0, inplace=True)

    return df_kpis


### - Função para criar o df_kpi_vendedor
def create_kpi_vendedor_df_v2(kpi_period_:str, df_metas: pd.DataFrame, date: datetime.datetime, df_vendas_pdv_: pd.DataFrame) -> pd.DataFrame:
    maskmeta, maskvenda = create_masks_meta_vendedor(periodo=kpi_period_, df_vendas_=df_vendas_pdv_, df_metas=df_metas, date=date)
    AGG_DICT = { 
        "discount": "sum", 
        "netValue": "sum", 
        "cost": "sum" 
    }

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["cpf_vendedor_inteiro", "year",],
        "Mês": ["cpf_vendedor_inteiro", "year", "month", ],
        "Dia": ["cpf_vendedor_inteiro", "year", "month", "day", ],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period_]

    df_kpis: pd.DataFrame = df_vendas_pdv_[maskvenda].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["fiscalsParentOrder"].nunique().reset_index().rename(columns={"fiscalsParentOrder": "tkts"}),
        how="left",
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskvenda].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    metas_df = df_metas[maskmeta].copy()
    metas_df.rename(columns={"distributorId": "loja_esforco"}, inplace=True)
    
    # dfmodificado = df_kpis.merge(
    #     metas_df,
    #     how="left"
    # )
    
    df_kpis = df_kpis.merge(
        metas_df.groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_.groupby(group_by_columns)["invoice_date"]
        .nunique()
        .reset_index()
        .rename(columns={"invoice_date": "days_of_work"}),
        how="left",
    )

    if kpi_period_ == "Dia":
        df_kpis["Dia"] = reportDate.day

    if kpi_period_ == "Mês":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
    else:
        df_kpis["tkts_per_day"] = df_kpis["tkts"]

    df_kpis["type"] = kpi_period_

    df_kpis["mkp"] = (df_kpis["netValue"] / df_kpis["cost"]).round(2)

    df_kpis["tkt_medio"] = df_kpis["netValue"] / df_kpis["tkts"].round(2)
    df_kpis["meta_percentual"] = (df_kpis["netValue"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount"] / df_kpis["netValue"]) * 100).round(2)
    
    df_kpis.replace({'mkp': [np.inf, -np.inf]}, 0, inplace=True)

    return df_kpis

In [803]:
### - Função para criar o df_kpi_loja
def create_kpi_loja_distributor_df(kpi_period_:str, df_metas: pd.DataFrame, date: datetime.datetime, df_vendas_pdv_: pd.DataFrame, loja_esforco:bool) -> pd.DataFrame:
    maskMeta, maskVendas = create_masks_meta_loja(periodo=kpi_period_, df_vendas_= df_vendas_pdv_, df_metas=df_metas, date=date)
    AGG_DICT = {"discount": "sum", "netValue": "sum", "cost": "sum"}

    if loja_esforco:
        GROUP_BY_COLUMNS_DICT = {
            "Ano": ["loja_esforco", "year"],
            "Mês": ["loja_esforco", "year", "month"],
            "Dia": ["loja_esforco", "year", "month", 'day'],
        }
    else:
        GROUP_BY_COLUMNS_DICT = {
            "Ano": ["distributorId", "year"],
            "Mês": ["distributorId", "year", "month"],
            "Dia": ["distributorId", "year", "month", 'day'],
        }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period_]

    df_kpis: pd.DataFrame = df_vendas_pdv_[maskVendas].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskVendas].groupby(group_by_columns)["fiscalsParentOrder"].nunique().reset_index().rename(columns={"fiscalsParentOrder": "tkts"}),
        how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv_[maskVendas].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    metas_df = df_metas[maskMeta].copy()
    
    if loja_esforco:
        metas_df.rename(columns={"distributorId": "loja_esforco"}, inplace=True)
    
    dfmodificado = df_kpis.merge(
        metas_df,
        how="left"
    )
    
    df_kpis = df_kpis.merge(
        dfmodificado.groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )
    
    if kpi_period_ == "Dia":
        df_kpis["Dia"] = date.day

    if kpi_period_ == "Dia":
        df_kpis["tkts_per_day"] = df_kpis["tkts"]
    if kpi_period_ == "Mês":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / date.day
    if kpi_period_ == "Ano":
        df_kpis["tkts_per_day"] = df_kpis["tkts"] / date.timetuple().tm_yday 
        
    df_kpis["type"] = kpi_period_
    df_kpis["mkp"] = (df_kpis["netValue"] / df_kpis["cost"]).round(2)
    df_kpis["tkt_medio"] = (df_kpis["netValue"] / df_kpis["tkts"]).round(2)
    df_kpis["meta_percentual"] = (df_kpis["netValue"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = (df_kpis["amount"] / df_kpis["tkts"]).round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount"] / df_kpis["netValue"]) * 100).round(2)

    df_kpis.replace({'mkp': [np.inf, -np.inf]}, 0, inplace=True)
    
    return df_kpis


In [806]:
### - Funções para criar uma lista de kpi por periodo de tempo (Dia | Mês | Ano)
def create_kpi_loja_esforco_period_list():
    df_loja_pdv, df_distributor_pdv = create_vendas_pdv(local_df_fato_vendas)
    kpis_loja_dict_ = {}
    kpis_distributor_dict_ = {}
    
    for kpi_period_ in ["Dia", "Mês", "Ano"]:
        kpis_loja_dict_[f"kpis_loja_{kpi_period_.lower()}"] = create_kpi_loja_distributor_df(
            kpi_period_=kpi_period_, 
            df_vendas_pdv_=df_loja_pdv, 
            df_metas=local_df_metas,
            date=reportDate,
            loja_esforco=True,
        )
        kpis_distributor_dict_[f"kpis_loja_{kpi_period_.lower()}"] = create_kpi_loja_distributor_df(
            kpi_period_= kpi_period_, 
            df_vendas_pdv_= df_distributor_pdv, 
            df_metas=local_df_metas, 
            date=reportDate,
            loja_esforco=False,
        )
    
    df_kpis_loja_list = pd.concat(list(kpis_loja_dict_.values()))
    df_kpis_distributor_list = pd.concat(list(kpis_distributor_dict_.values()))
    
    return df_kpis_loja_list, df_kpis_distributor_list

def create_kpi_vendedor_period_list():
    df_vendedor_pdv, df = create_vendas_pdv(local_df_fato_vendas)
    kpis_vendedor_dict_ = {}
        
    for kpi_period_ in ["Dia", "Mês", "Ano"]:
        kpis_vendedor_dict_[f"kpis_vendedor_{kpi_period_.lower()}"] = create_kpi_vendedor_df(kpi_period_=kpi_period_, df_vendas_pdv_=df_vendedor_pdv, df_metas=local_df_metas, date=reportDate)

    df_kpis_vendedor_list = pd.concat(list(kpis_vendedor_dict_.values()))
    
    return df_kpis_vendedor_list


In [807]:
### - Função para corrigir vendas vazias para o dia
def fix_kpi_LojaList(id_loja:str, date: datetime.date, df_loja_list: pd.DataFrame, df_metas: pd.DataFrame, loja_esforco:bool):
    df = df_loja_list.copy()
    df = df.fillna(0).copy()

    tipoMetaMask = df_metas['tipo_meta'] == 'Meta Loja PDV'
    lojaMetaMask = df_metas['distributorId'] == id_loja
    df_metas = df_metas[lojaMetaMask & tipoMetaMask].copy()
    

    if loja_esforco:
        lojaMask = df['loja_esforco']==id_loja
    else:
        lojaMask = df['distributorId']==id_loja

    typeDayMask = df['type'] == 'Dia'
    typeMonthMask = df['type'] == 'Mês'
    typeYearMask = df['type'] == 'Ano'
    yearMask = df['year'] == date.year
    monthMask = df['month'] == date.month
    dayMask = df['day'] == date.day

    typeYear = lojaMask & typeYearMask & yearMask
    typeMonth = lojaMask & yearMask & typeMonthMask & monthMask
    typeDay = lojaMask & yearMask & typeDayMask & monthMask & dayMask

    dfAno = df[typeYear]
    dfMes = df[typeMonth]
    dfDia = df[typeDay]

    if dfDia.empty:
        dfDia.loc['replace'] = 0
        dataMetaMask = df_metas['date'] == date
        df_metasDia = df_metas[dataMetaMask].copy()
        meta = df_metasDia['meta'].max()
        dfDia['meta'] = meta
        dfDia.loc[:,'loja_esforco'] = id_loja
        dfDia.loc[:,'year'] = date.year
        dfDia.loc[:,'month'] = date.month
        dfDia.loc[:,'day'] = date.day
        dfDia.loc[:,'type'] = 'Dia'
    
    if dfMes.empty:
        dfMes.loc['replace'] = 0
        maskMetaYear = df_metas['year'] == date.year
        maskMetaMes = df_metas['month'] == date.month
        df = df_metas[maskMetaYear & maskMetaMes].copy()
        meta = df.groupby(['distributorId']).agg({'meta':'sum'})['meta'].max()
        dfMes['meta'] = meta
        dfMes['loja_esforco'] = id_loja
        dfMes['year'] = date.year
        dfMes['month'] = date.month
        dfMes['type'] = 'Mês'
    
    if dfAno.empty:
        dfAno.loc['replace'] = 0
        maskMetaYear = df_metas['year'] == date.year
        df = df_metas[maskMetaYear].copy()
        meta = df.groupby(['distributorId']).agg({'meta':'sum'})['meta'].max()
        dfAno['meta'] = meta
        dfAno['loja_esforco'] = id_loja
        dfAno['year'] = date.year
        dfAno['type'] = 'Ano'
        

    return pd.concat([dfAno, dfMes, dfDia]).reset_index()


In [808]:
### - Criação dos kpis por periodo 
df_kpis_loja_list, df_kpis_distributor_list = create_kpi_loja_esforco_period_list()
df_kpis_vendedor_list = create_kpi_vendedor_period_list()

In [819]:
### - Muda o número do mes por Jan/Fev/Mar....
def fixMonthToPtMes(month:str) -> str:
    if month == "1":
        return "Jan"
    if month == "2":
        return "Fev"
    if month == "3":
        return "Mar"
    if month == "4":
        return "Abr"
    if month == "5":
        return "Mai"
    if month == "6":
        return "Jun"
    if month == "7":
        return "Jul"
    if month == "8":
        return "Ago"
    if month == "9":
        return "Set"
    if month == "10":
        return "Out"
    if month == "11":
        return "Nov"
    if month == "12":
        return "Dev"
    

In [820]:
### - Cria, limpa e formata o html que irá no corpo do email
def create_html_text_for_loja(id_loja:str, date: datetime.datetime, df_loja: pd.DataFrame, loja_esforco:bool) -> str:
    cssTable="table { border-collapse:collapse; }"
    cssThTd="th,td { padding: 8px; }tr:nth-child(even) { background-color: #f2f2f2; } tr:nth-child(odd) { background-color: #fafafa; } "
    placeholder = '#%TITLE%#'
    htmlValueStore = """
<html>
<head><style>{2} {3}</style></head>
<body>
<h2>{1}</h2>
{0}
</body>
<br/><br/>
</html>""".format(
    df_loja.to_html(), 
    placeholder,
    cssTable,
    cssThTd,
)
    
    htmlValueStore = htmlValueStore.replace(" ", "")
    htmlValueStore = htmlValueStore.replace("""
<trstyle="text-align:right;">
<th></th>
<th>0</th>
<th>1</th>
<th>2</th>
</tr>
</thead>
<tbody>
<tr>
<th>Período</th>
<td>Dia</td>
<td>Mês</td>
<td>Ano</td>
</tr>
""",
"""
<tr style="text-align:center;>
<th> Visão por esforço de venda</th>
</tr>
<tr style="text-align:center;font-size:14px; font-weight:bold;">
<th></th>
<th style="background:#0288c0; color:#FFF; padding: 5px 20px;">Dia</th>
<th style="background:#0288c0; color:#FFF; padding: 5px 20px;">Mês</th>
<th style="background:#0288c0; color:#FFF; padding: 5px 20px;">Ano</th>
</thead>
<tbody style="font-size:12px;">
""")
    htmlValueStore = htmlValueStore.replace(
        '<tableborder="1"class="dataframe">',
        '<table border="1" class="dataframe" style="text-align:center;">'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>Venda</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Venda</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>Meta</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Meta</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>Meta%</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Meta %</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>Tickets/Dia</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Tickets / Dia</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>PeçasporAtendimento</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Peças por Atendimento</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>TicketMédio</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Ticket Médio</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>DescontoMédio%</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Desconto Médio %</th>'
    )
    htmlValueStore = htmlValueStore.replace(
        '<th>Markup</th>',
        '<th style="background:#0288c0; color:#FFF;padding: 0 10px;font-size:14px;">Markup</th>'
    )
    
    if loja_esforco:
        title = 'Visão por esforço de venda'
    else:
        title = 'Visão por faturamento'
    htmlValueStore = htmlValueStore.replace(placeholder, title)
    htmlValueStore = htmlValueStore.replace('R$', 'R$ ')
    
    return htmlValueStore

def create_html_text_for_vendedor_dia(id_loja:str, date: datetime.datetime, df_vendedor_dia: pd.DataFrame,) -> str:
    cssTable="table { border-collapse:collapse; }"
    cssThTd="th,td { padding: 8px; }tr:nth-child(even) { background-color: #f2f2f2; } tr:nth-child(odd) { background-color: #fafafa; } "
    placeholder = '#%TITLE%#'
    htmlDiaValue = """
<html>
<head><style>{2} {3}</style></head>
<body>
<h3>{1}</h3>
    {0}
</body>
<br/><br/>
</html>
""".format(
    df_vendedor_dia.to_html(), 
    placeholder,
    cssTable,
    cssThTd,
)
    htmlDiaValue = htmlDiaValue.replace(" ","")
    htmlDiaValue = htmlDiaValue.replace(
        """
<tableborder="1"class="dataframe">
<thead>
<trstyle="text-align:right;">
<th></th>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
</tr>
<tr>
<th>0</th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<th>Vendedor</th>
<td>AcumuladonoDia</td>
<td>Homogeneidade</td>
<td>MetadoDia</td>
<td>Meta%</td>
<td>Markup</td>
<td>Tickets</td>
<td>PeçasporAtendimento</td>
<td>TicketMédio</td>
</tr>""",
        """
<table border="1" class="dataframe" style="text-align:center;">
<thead>
<tr style="text-align:center; background:#0088d1; color:#FFF;font-size:14px;font-weight: bold;">
<th style="padding: 6px 12px;background:#0088d1;">Vendedor</th>
<th style="padding: 6px 12px;background:#0088d1;">Acumulado <br>no Dia</th>
<th style="padding: 6px 12px;background:#0088d1;">Homogeneidade</th>
<th style="padding: 6px 12px;background:#0088d1;">Meta <br>do Dia</th>
<th style="padding: 6px 12px;background:#0088d1;">Meta %</th>
<th style="padding: 6px 12px;background:#0088d1;">Markup</th>
<th style="padding: 6px 12px;background:#0088d1;">Tickets</th>
<th style="padding: 6px 12px;background:#0088d1;">Peças <br> por Atendimento</th>
<th style="padding: 6px 12px;background:#0088d1;">Ticket Médio</th>
</tr>
</thead>
<tbody style="font-weight: 300;font-size:12px;">"""
    )
    
    htmlDiaValue = htmlDiaValue.replace('(@!_)',' ')
    htmlDiaValue = htmlDiaValue.replace('R$','R$ ')
    strDate = date.strftime('%d-%m-%Y')
    htmlDiaValue = htmlDiaValue.replace(placeholder, f'Esforço de venda - Vendedores - Data: {strDate}')
    
    return htmlDiaValue

def create_html_text_for_vendedor_mes(date: datetime.datetime, df_vendedor_mes: pd.DataFrame ) -> str:
    cssTable="table { border-collapse:collapse; }"
    cssThTd="th,td { padding: 8px; }tr:nth-child(even) { background-color: #f2f2f2; } tr:nth-child(odd) { background-color: #fafafa; } "
    placeholder = '#%TITLE%#'
    htmlMesValue = """
    <html>
    <head><style>{2} {3}</style></head>
    <body>
    <h3>{1}</h3>
        {0}
    </body>
    <br/><br/>
    </html>
    """.format(
        df_vendedor_mes.to_html(), 
        placeholder, 
        cssTable, 
        cssThTd
    )
    
    htmlMesValue = htmlMesValue.replace(" ", "")
    htmlMesValue = htmlMesValue.replace(
"""<tableborder="1"class="dataframe">
<thead>
<trstyle="text-align:right;">
<th></th>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
<th>9</th>
</tr>
<tr>
<th>0</th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
<th></th>
</tr>
</thead>
<tbody>
<tr>
<th>Vendedor</th>
<td>DiasTrabalhados</td>
<td>AcumuladonoMês</td>
<td>Homogeneidade</td>
<td>MetadoMês</td>
<td>Meta%</td>
<td>Markup</td>
<td>Tickets/Dia</td>
<td>PeçasporAtendimento</td>
<td>TicketMédio</td>
</tr>""",

"""<table border="1" class="dataframe" style="text-align:center;">
<thead>
<tr style="text-align:center; background:#0088d1; color:#FFF;font-weight: bold;font-size:14px;">
<th style="padding: 6px 10px;">Vendedor</th>
<th style="padding: 6px 10px;">Dias <br> Trabalhados</th>
<th style="padding: 6px 10px;">Acumulado <br> no Mês</th>
<th style="padding: 6px 10px;">Homogeneidade</th>
<th style="padding: 6px 10px;">Metado Mês</th>
<th style="padding: 6px 10px;">Meta %</th>
<th style="padding: 6px 10px;">Markup</th>
<th style="padding: 6px 10px;">Tickets / Dia</th>
<th style="padding: 6px 10px;">Peças <br> por Atendimento</th>
<th style="padding: 6px 10px;">Ticket Médio</th>
</tr>
</thead>
<tbody style=";font-weight: 300;font-size:12px;">""")
    htmlMesValue = htmlMesValue.replace('(@!_)', ' ')
    htmlMesValue = htmlMesValue.replace('R$', 'R$ ')
    strDateMes = fixMonthToPtMes(str(date.month))
    strDateAno = date.year
    htmlMesValue = htmlMesValue.replace(placeholder, f'Esforço de venda - Vendedores - Mês: {strDateMes}/{strDateAno}')
    
    return htmlMesValue

In [804]:
### - Função para formatar o kpi_loja
def fix_idx(value: str)->int:
    if value == "Dia":
        return 0
    elif value == "Mês":
        return 1
    elif value == "Ano":
        return 2
    
def format_kpi_loja(id_loja: str, df_kpi:pd.DataFrame, date: datetime.datetime, loja_esforco:bool):
    if loja_esforco:
        mask = df_kpi["loja_esforco"] == id_loja
    else:
        mask = df_kpi["distributorId"] == id_loja
        
    df_store = df_kpi[mask].copy()
    # df_store = df_kpi[mask].fillna(0).copy()
    maskAno = df_store['year'] == date.year
    df_store = df_store[maskAno]
    
    df_store_current_month = df_store.loc[df_store.groupby(["type"])["month"].idxmax()].drop_duplicates()
    df_store_current_month.drop(columns=['discount_percentual'], inplace=True)
    rename_dict = {
        "netValue": "Venda",
        "meta": "Meta",
        "meta_percentual": "Meta %",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # "discount_percentual": "Desconto Médio %",
        "type": "Período",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Venda": format_money,
        "Meta": format_money,
        "Meta %": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # "Desconto Médio %": format_percentage,
    }

    df_store_current_month['idx_helper'] = df_store_current_month['type'].apply(fix_idx)
    df_store_current_month.sort_values(by='idx_helper', ascending=True, inplace=True)
    df_store_current_month.drop(columns='idx_helper')
    
    df_store_formated = df_store_current_month[rename_dict.keys()].round(2).rename(columns=rename_dict)
    
    for key, format in format_dict.items():
        df_store_formated[key] = df_store_formated[key].map(format.format)

        if format == format_money:
            df_store_formated[key] = df_store_formated[key].astype(str).str.replace(",", ".")

    try:
        df_store_formated = df_store_formated.set_index("Período").reset_index().T
    except KeyError:
        df_store_formated = pd.DataFrame()
    
    return df_store_formated

In [805]:
### - Função para formatar o kpi_vendedor (dia | mes)
def format_kpi_vendedor(id_loja:str, kpi_df_vendedor:pd.DataFrame, vendedor_loja_df:pd.DataFrame, df_vendedores: pd.DataFrame,periodo: str, date: datetime.datetime):
    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"
    
    mask = kpi_df_vendedor["loja_esforco"] == id_loja
    df_vendedor = kpi_df_vendedor[mask].reset_index(drop=True)
    df_vendedor["Mês"] = df_vendedor["month"].fillna(0)

    mask_mes = df_vendedor["Mês"] == date.month
    mask_ano = df_vendedor["year"] == date.year
    
    if periodo == "Dia":
        mask_type = df_vendedor["type"].isin(["Dia"])
        mask_dia = df_vendedor['Dia'] == date.day
        mask_completa = mask_type & mask_dia & mask_mes & mask_ano
    else:
        mask_type = df_vendedor["type"].isin(["Mês"])
        mask_completa = mask_type & mask_mes & mask_ano
        
    df_vendedor_filtro: pd.DataFrame = df_vendedor[mask_completa].reset_index(drop=True)
    
    df_vendedor_filtro["normalized_net_value"] = (
        df_vendedor_filtro["netValue"] * 100 / round(df_vendedor_filtro["netValue"].max(), 2)
    )
    
    df_vendedor_filtro["net_value_share"] = (
        df_vendedor_filtro["netValue"] * 100 / round(df_vendedor_filtro["netValue"].max(), 2)
    )
    
    df_vendedor_filtro = merge_kpiVendedor_nome(
        vendedor_loja=vendedor_loja_df, 
        df_vendedores=df_vendedores, 
        df_kpiVendedor_filtro=df_vendedor_filtro, 
        id_loja=id_loja
    )
    
    df_vendedor_filtro.drop(columns='discount_percentual', inplace=True)
    
    if periodo == "Dia":
        rename_dict = {
            "vendedor": "Vendedor",
            "netValue": "Acumulado no Dia",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Dia",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            # "discount_percentual": "Desconto Médio %",
        }
        
        format_dict = {
            "Acumulado no Dia": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Dia": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets": "{:,.0f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            # "Desconto Médio %": format_percentage,
        }
        
        df_vendedor_formatado = (
            df_vendedor_filtro[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .rename(columns=rename_dict)
        )
        
    else:
        rename_dict = {
            "vendedor": "Vendedor",
            "days_of_work": "Dias Trabalhados",
            "netValue": "Acumulado no Mês",
            "normalized_net_value": "Homogeneidade",
            "meta": "Meta do Mês",
            "meta_percentual": "Meta %",
            "mkp": "Markup",
            "tkts_per_day": "Tickets/Dia",
            "PA": "Peças por Atendimento",
            "tkt_medio": "Ticket Médio",
            # "discount_percentual": "Desconto Médio %",
        }
        
        format_dict = {
            "Acumulado no Mês": format_money,
            "Homogeneidade": format_percentage,
            "Meta do Mês": format_money,
            "Meta %": format_percentage,
            "Markup": "{:,.2f}",
            "Tickets/Dia": "{:,.1f}",
            "Peças por Atendimento": "{:,.1f}",
            "Ticket Médio": format_money,
            # "Desconto Médio %": format_percentage,
        }

        df_vendedor_formatado = (
            df_vendedor_filtro[rename_dict.keys()]
            .sort_values("normalized_net_value", ascending=False)
            .round(2)
            .rename(columns=rename_dict)
        )

    for key, format in format_dict.items():
        df_vendedor_formatado[key] = df_vendedor_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_formatado[key] = df_vendedor_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_formatado = df_vendedor_formatado.reset_index(drop=True).T.reset_index().T

    return df_vendedor_formatado.set_index(0)

In [812]:
### - Formatando todos os kpis
kpis_loja_formated_dict_ = {}
kpis_distributor_formated_dict_ = {}

kpis_vendedor_dia_formated_dict_ = {}
kpis_vendedor_mes_formated_dict_ = {}

for loja in storesList:
    kpis_loja_formated_dict_[f"{loja}"] = format_kpi_loja(
        id_loja=loja, 
        df_kpi=fix_kpi_LojaList(
                id_loja=loja, 
                date=reportDate, 
                df_loja_list=df_kpis_loja_list, 
                df_metas=local_df_metas, 
                loja_esforco=True
            ),
        date=reportDate,
        loja_esforco=True,
    )
    
    kpis_distributor_formated_dict_[f"{loja}"] = format_kpi_loja(
        id_loja=loja, 
        df_kpi=fix_kpi_LojaList(
            id_loja=loja, 
            date=reportDate, 
            df_loja_list=df_kpis_distributor_list, 
            df_metas=local_df_metas, 
            loja_esforco=False
        ),
        date=reportDate,
        loja_esforco=False,
    )
    
    kpis_vendedor_dia_formated_dict_[f"{loja}"] = format_kpi_vendedor(
        id_loja=loja,
        kpi_df_vendedor=df_kpis_vendedor_list,
        vendedor_loja_df=local_df_vendedor_loja,
        df_vendedores=local_df_vendedores,
        periodo='Dia',
        date=reportDate
    )
    
    kpis_vendedor_mes_formated_dict_[f"{loja}"] = format_kpi_vendedor(
        id_loja=loja,
        kpi_df_vendedor=df_kpis_vendedor_list,
        vendedor_loja_df=local_df_vendedor_loja,
        df_vendedores=local_df_vendedores,
        periodo='Mês',
        date=reportDate
    )


In [816]:
### - Definindo conteúdo html e recipients
fullHtml = ""
partialHtml = ""
for store in storesList:
    lojaDF:pd.DataFrame = kpis_loja_formated_dict_[f'{store}']
    distributorDF:pd.DataFrame = kpis_distributor_formated_dict_[f'{store}']
    diaDF:pd.DataFrame = kpis_vendedor_dia_formated_dict_[f'{store}']
    mesDF:pd.DataFrame = kpis_vendedor_mes_formated_dict_[f'{store}']
    
    fullHtml += """
<html>
<head>
</head>
<body>
<h1 style="margin:0;font-size:28px;"> Relatorio loja - {0} | Data: {1}</h1>
    """.format(store, reportDate.strftime('%d-%m-%Y'))
    if not distributorDF.T.empty:
        distribuidor = create_html_text_for_loja(
            id_loja=store, 
            date=reportDate, 
            df_loja=distributorDF,
            loja_esforco=False,
        )
    if not lojaDF.T.empty:
        sttingloja = create_html_text_for_loja(
            id_loja=store, 
            date=reportDate, 
            df_loja=lojaDF,
            loja_esforco=True,
        )
    
    partialHtml += """
<table border="0" style="text-align:center;">
  <tr>
    <th></th>
    <th style="padding:0 30px;color:#FFF;background:#FFF;">_</th>
    <th></th>
  </tr>
  <tr>
    <td>{0}</td>
    <td></td>
    <td>{1}</td>
  </tr>
</table>
    """.format(distribuidor, sttingloja)
    
    fullHtml += partialHtml
    fullHtml += create_html_text_for_vendedor_dia(
        id_loja=store, 
        date=reportDate, 
        df_vendedor_dia=diaDF, 
    )
    
    fullHtml += create_html_text_for_vendedor_mes(
        date=reportDate, 
        df_vendedor_mes=mesDF, 
    )
relatorioDiaStr = reportDate.strftime("%d-%m-%Y")
emailRecipients = ['joao.garcia@ammovarejo.com.br']

In [817]:
import os
import smtplib
from dataclasses import dataclass
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import Any, Callable, Dict, List, Optional, Union
from commom.logger import logger
from commom.base_classes.base_sender import BaseMessenger
from innovation_messenger.config import config

@dataclass
class IEmailProperties:
    subject: str
    body: str
    recipient: Union[str, List[str]]
    file_name: Optional[str] = None


def send_email_html(email_properties: IEmailProperties) -> bool:
    message = MIMEMultipart()
    message["Subject"] = email_properties.subject
    message["From"] = 'inovacao@ammovarejo.com.br'
    messageTo: str

    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient

    message["To"] = messageTo
    html_part = MIMEText(email_properties.body, 'html')
    message.attach(html_part)
    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient
        
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        try:
            server.login(user='inovacao@ammovarejo.com.br', password='inovaAM!05')
            server.sendmail('inovacao@ammovarejo.com.br', email_properties.recipient, message.as_string())
            return True
        except Exception as e:
            logger.error(e)
            return False


email_properties = IEmailProperties(
    subject=f'Teste - Loja - {storesList[0]} - Dia {relatorioDiaStr}',
    recipient=emailRecipients,
    body=fullHtml,
)

send_email_html(
  email_properties=email_properties,
)


True

In [818]:
local_df_vendedor_loja
local_df_vendedores

,cpf,name,employeePositionId,Cadastro,cargo,ativo,Desligamento
0,37235653876,Pedro de Oliveira,28,NaT,GERENTE DE LOJA,Ativo,NaT
1,31682467864,Eduardo Nardi,28,NaT,GERENTE DE LOJA,Ativo,NaT
2,04913909398,Jackson Miranda,28,NaT,GERENTE DE LOJA,Ativo,NaT
3,45681862852,Didimo Nanine,147,NaT,VENDEDOR (A),Ativo,NaT
4,04159646603,ANDREIA ANDRADE BARCELOS,28,2019-10-07,GERENTE DE LOJA,Ativo,NaT
...,...,...,...,...,...,...,...
5145,01890618390,SIMONE RAMOS FERREIRA,52,2023-04-08,AUXILIAR DE LOJA,Ativo,NaT
5146,15445278611,Andressa Gabriela dos Santos,147,2023-04-08,VENDEDOR (A),Ativo,NaT
5147,01875907025,JENNIFER MARIA PISTOR,147,2023-12-20,VENDEDOR (A),Ativo,NaT
5148,04855036978,claudia do carmo de oliveira,147,2024-09-01,VENDEDOR (A),Ativo,NaT
